In [ ]:
PATHS MIGHT NEED TO BE FIXED TO RUN THESE CELLS

In [ ]:
# import pandas as pd
# import pickle

# final = pd.read_csv("spotify_dataset/final.csv")
# clean = pd.read_csv("spotify_dataset/clean.csv")

# # Merge on track_id
# df = pd.merge(
#     clean, final[["track_id", "track_name", "album_name"]], on="track_id", how="left"
# )

# df.head()
# print(len(df))

# df.to_csv("./merged_song_dataset.csv")


In [ ]:
# To JSON
import pandas as pd

df = pd.read_csv("dataset/millionSongs.csv")

mesh = df[["id", "name", "danceability", "energy", "year"]].copy()

mesh["x"] = mesh["danceability"] * 2 - 1
mesh["y"] = mesh["energy"] * 2 - 1

mesh = mesh[["id", "name", "x", "y", "energy", "year"]]

mesh.to_json("songs_mesh.json", orient="records")



In [ ]:
# TO BINARY
import json
import numpy as np

with open("dataset/songs_mesh.json") as f:
    data = json.load(f)

x = np.array([song["x"] for song in data], dtype=np.float32)
y = np.array([song["y"] for song in data], dtype=np.float32)

coords = np.column_stack([x, y]).astype(np.float32)

coords.tofile("dataset/coords.bin")

ids = np.array([i for i in range(len(data))], dtype=np.uint32)
ids.tofile("ids.bin")

meta = [{"id": d["id"], "name": d["name"]} for d in data]
with open("meta.json", "w") as f:
    json.dump(meta, f)

In [ ]:
NEW APPROACH JUST TESTING

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("..\dataset\millionSongs.csv")

x = (df["danceability"].values * 2 - 1).astype(np.float32)
y = (df["energy"].values * 2 - 1).astype(np.float32)

# Energy stays 1 to 1 for the color mapping in the Fragment Shader
energy = df["energy"].values.astype(np.float32)

# This creates an array: [x1, y1, e1, x2, y2, e2, ...]
# This matches gl.vertexAttribPointer(loc, 3, gl.FLOAT, ...)
coords = np.column_stack([x, y, energy]).astype(np.float32)

coords.tofile("..\dataset\coords_new.bin")

print(f"Binary file created with {len(df)} songs.")
print(f"Total floats: {coords.size}")

In [ ]:
TESTING ONLY 1 SONG

In [ ]:
import pandas as pd

file_path = r'..\dataset\test.csv'

df = pd.read_csv(file_path, nrows=1)
print(df.iloc[0])

In [ ]:
x = (df["danceability"].values * 2 - 1).astype(np.float32)
y = (df["energy"].values * 2 - 1).astype(np.float32)

# Energy stays 0 to 1 for the color mapping in the Fragment Shader
energy = df["energy"].values.astype(np.float32)

# This creates an array: [x1, y1, e1, x2, y2, e2, ...]
# This matches gl.vertexAttribPointer(loc, 3, gl.FLOAT, ...)
coords = np.column_stack([x, y, energy]).astype(np.float32)

coords.tofile("..\dataset\coords_one.bin")

print(f"Binary file created with {len(df)} songs.")
print(f"Total floats: {coords.size}")

In [ ]:
TESTING 0,1000 world mapping

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("..\dataset\millionSongs.csv")

# Map danceability (0..1) to 0..1000
x = (df["danceability"].values * 1000).astype(np.float32)
# Map energy (0..1) to 0..1000
y = (df["energy"].values * 1000).astype(np.float32)

# Energy stays 1 to 1 for the color mapping in the Fragment Shader
energy = df["energy"].values.astype(np.float32)

# This creates an array: [x1, y1, e1, x2, y2, e2, ...]
# This matches gl.vertexAttribPointer(loc, 3, gl.FLOAT, ...)
coords = np.column_stack([x, y, energy]).astype(np.float32)

coords.tofile("..\dataset\coords_new.bin")

print(f"Binary file created with {len(df)} songs.")
print(f"Total floats: {coords.size}")

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("..\dataset\millionSongs.csv")

x_coords = (df["danceability"].values * 1000).astype(np.float32)
y_coords = (df["energy"].values * 1000).astype(np.float32)
energy_values = df["energy"].values.astype(np.float32)

grid_x = (x_coords // 100).clip(0, 9).astype(int)
grid_y = (y_coords // 100).clip(0, 9).astype(int)

df['box_id'] = [f"box_{gx}_{gy}" for gx, gy in zip(grid_x, grid_y)]

coords = np.column_stack([x_coords, y_coords, energy_values]).astype(np.float32)
coords.tofile("..\dataset\coords_new1.bin")

df.to_csv("..\dataset\millionSongs_with_boxes.csv", index=False)

print(f"Binary file created with {len(df)} songs.")
print("Box IDs assigned and CSV updated.")

In [ ]:
TESTING/COMPAIRING BINARIES WITH CSV

In [ ]:
import numpy as np
import pandas as pd

baked = np.fromfile("../dataset/coords_baked.bin", dtype=np.float32).reshape(-1, 3)
baked_sample_energy = baked[:5, 2] # Get energy of first 5 points

df = pd.read_csv("../dataset/millionSongs.csv", nrows=5)
csv_sample_energy = df['energy'].values

print("Baked Energy Sample:", baked_sample_energy)
print("CSV Energy Sample:  ", csv_sample_energy)

# Check if they match
if np.allclose(baked_sample_energy, csv_sample_energy, atol=1e-5):
    print("SUCCESS: Index alignment confirmed!")
else:
    print("ERROR: Alignment mismatch. The CSV rows do not match the BIN points.")